In [1]:
import urllib3
import datetime
urllib3.disable_warnings()

In [12]:
import numpy as np
import json
from pathlib import Path
import os
import requests
from datetime import datetime
import datetime as dtt

def get_returns(candle):
    rendements = []
    for j in range(len(candle) - 1):
        j1 = candle[j]
        j2 = candle[j + 1]
        cash_flow = 0
        if "cash_flow" in j2:
            cash_flow = j2["cash_flow"]
        r = (j2["close"] - j1["close"] + cash_flow)/j1["close"]
        rendements.append(r)
    return np.array(rendements)
    
def get_vol(candle):
    closes = np.array([x["open"] for x in candles])
    return get_returns(candles).var() * 100
    
def get_volatility(id, start, end):
    param = {"ratio": [18], "asset": [id], "start_date": start, "end_date": end}
    s = requests.Session()
    s.auth = ('epita_user_6', 'dolphin39833')
    s.body = param
    r = s.post("https://dolphin.jump-technology.com:3389/api/v1/ratio/invoke", data = json.dumps(param), verify=False)
    r = r.json()
    print(r)
    
    
def pull_data():
    s = requests.Session()
    s.auth = ('epita_user_6', 'dolphin39833')
    r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset", verify=False)
    data = r.json()
    ids = [a["ASSET_DATABASE_ID"] for a in data]
    for i in ids:
#        print(i["value"])
        p = Path("../../data/raw/" + i["value"])
        if not p.exists():
            f = open("../../data/raw/" + i["value"], "w")
            r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset/" + i["value"] + "/quote", verify=False)
            candles = r.json()
            f.write(json.dumps(candles))
            f.close()
    
    
def dateInPeriod(date, start, end):
    #start = datetime.strptime(start, "%Y-%m-%d")
    #end = datetime.strptime(end, "%Y-%m-%d")
    if date >= start and date <= end:
        return True
    return False
    
    
def cleanData(path):
    fi = open(path, "r")
    data = fi.readlines()[0]
    data = json.loads(data)

    start_date = data[0]["date"]
    end_date = data[-1]["date"]
    
    cleaned = [[data[0]["date"], data[0]["close"]]]
    prev = datetime.strptime(data[0]["date"], "%Y-%m-%d")

    for e in data[1:]:
        diff = (datetime.strptime(e["date"], "%Y-%m-%d") - prev).days
        prev_val = cleaned[-1][1]
        for i in range(1, diff + 1):
            prev = prev + dtt.timedelta(days=1)
            tmp = [datetime.strftime(prev, "%Y-%m-%d"), prev_val + i * (e["close"] - prev_val) / diff]
            cleaned.append(tmp)
        prev = datetime.strptime(e["date"], "%Y-%m-%d")
        
    fo = open("../../data/cleaned/" + path.split("/")[-1], "w")
    fo.write(json.dumps(cleaned))
    
def getMatrix(date_start, date_end):
    mat = []
    for root, dirs, files in os.walk("../../data/"):
        for f in files:
            if not f.startswith("."):
                fi = open(os.path.join(root, f))
                v = fi.readlines()[0]
                v = json.loads(v)
                ds = datetime.strptime(v[0]["date"], "%Y-%m-%d")
                de = datetime.strptime(v[-1]["date"], "%Y-%m-%d")
                if ds <= date_start and de >= date_end:
                    v = cleanData([[x["date"], float(x["close"])] for x in v], date_start, date_end)
                    v = np.array(v)
                    mat.append(v)
                #else:
                #    print("Can't take file ", f, " because starts at ", ds, " and end at ", de)
                fi.close()
               
    return np.array(mat)


#pull_data()
    
#def volatility(candle):
#def sharpe(candle):

In [ ]:
import requests

s = requests.Session()
s.auth = ('epita_user_6', 'dolphin39833')
r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset", verify=False)
data = r.json()
ids = [a["ASSET_DATABASE_ID"] for a in data]
i = ids[0]
r = s.get("https://dolphin.jump-technology.com:3389/api/v1/asset/" + i["value"] + "/quote", verify=False)
candles = r.json()

In [16]:
pull_data()

In [17]:
for root, dirs, files in os.walk("../../data/raw/"):
        for f in files:
            if not f.startswith("."):
                cleanData(os.path.join(root, f))